## Develop this to have entrylist with highest entry lowest and average

## maybe entry from the average trade script

In [34]:
import numpy as np 
import pandas as pd 
from datetime import date
#Data Source (Pandas Data Reader)
from pandas_datareader import data
import requests 
import json
from dateutil.relativedelta import relativedelta


#Data Source(Pandas Data Reader)


In [35]:
#entrylist =[['ANKR/USDT',0.07684],['AVAX/USDT',57],['WAXP/USDT',0.3818],['REEF/USDT',0.01346],['LINK/USDT',14.15],['AR/USDT',30],['ATOM/USDT',23],['MIR/USDT',2.343],['ALGO/USDT',0.909],['ZRX/USDT',0.90],['XTZ/USDT',2.856],['KSM/USDT',163],]
#entrylist =[['ANKRUSDT',0.07684],['AVAXUSDT',57],['WAXPUSDT',0.3818],['REEFUSDT',0.01131],['LINKUSDT',14.15],['ARUSDT',30],['ATOMUSDT',23],['MIRUSDT',2.343],['ALGOUSDT',0.909],['ZRXUSDT',0.60],['XTZUSDT',2.856],['KSMUSDT',163],['NEARUSDT',14],['WINUSDT',0.0003138],['CELRUSDT',0.05063],['DENTUSDT',0.002459],['LRCUSDT',0.7047],['CRVUSDT',3.529],['IMXUSDT',2.709],['SCRTUSDT',5],['RENUSDT',0.2547],['SUSHIUSDT',4.522],['CAKEUSDT',11.24],['XLMUSDT',0.2109],['HNTUSDT',30],['OMGUSDT',4.5],['LINAUSDT',0.024],['MIRUSDT',2.343],['FLUXUSDT',1.523]]
entrylist =[['ANKRUSDT',0.05415],['AVAXUSDT',57],['WAXPUSDT',0.3174],['REEFUSDT',0.00930],['LINKUSDT',14.15],['ARUSDT',30],['ATOMUSDT',23],['MIRUSDT',2.343],['ALGOUSDT',0.909],['ZRXUSDT',0.50],['XTZUSDT',2.856],['KSMUSDT',150],['NEARUSDT',10.320],['WINUSDT',0.0003138],['DENTUSDT',0.002459],['LRCUSDT',0.677],['CRVUSDT',2.12],['IMXUSDT',1.30],['SCRTUSDT',5],['RENUSDT',0.3047],['SUSHIUSDT',3.5],['CAKEUSDT',11.24],['XLMUSDT',0.2109],['HNTUSDT',30],['OMGUSDT',4.5],['LINAUSDT',0.01759],['FLUXUSDT',1.523],['SRMUSDT',1.8],['VTHOUSDT',0.0031],['ANTUSDT',5.616],['RAYUSDT',2.5],['ROSEUSDT',0.24055],['MOVRUSDT',51.6],['TFUELUSDT',0.22],['GLMRUSDT',4.5],['COTIUSDT',0.2277],['ONEUSDT',0.13171],['VETUSDT',0.05138],['RUNEUSDT',3.709],['CELRUSDT',0.034063],['ONTUSDT',0.4652],['GALAUSDT',0.24794],['GRTUSDT',0.3164],['DOTUSDT',16.31],['ZILUSDT',0.04410],['FILUSDT',20],['SCUSDT',0.015],['MATICUSDT',1.4],['FTMUSDT',1.0],['BANDUSDT',3.6],['1INCHUSDT',1.6],['AUDIOUSDT',1.0]]
exposurelist =[['ANKRUSDT',311],['AVAXUSDT',2850],['WAXPUSDT',527],['REEFUSDT',603],['LINKUSDT',6000],['ARUSDT',2450],['ATOMUSDT',1863],['MIRUSDT',85],['ALGOUSDT',2735],['ZRXUSDT',245],['XTZUSDT',1200],['KSMUSDT',7800],['NEARUSDT',1360],['WINUSDT',285],['DENTUSDT',385],['LRCUSDT',660],['CRVUSDT',323],['IMXUSDT',269],['SCRTUSDT',601],['RENUSDT',684],['SUSHIUSDT',2730],['CAKEUSDT',4000],['XLMUSDT',368],['HNTUSDT',999],['OMGUSDT',765],['LINAUSDT',205],['FLUXUSDT',224],['SRMUSDT',587],['VTHOUSDT',413],['ANTUSDT',200],['RAYUSDT',100],['ROSEUSDT',634],['MOVRUSDT',500],['TFUELUSDT',244],['GLMRUSDT',400],['COTIUSDT',500],['ONEUSDT',1000],['VETUSDT',500],['RUNEUSDT',750],['CELRUSDT',500],['ONTUSDT',400],['GALAUSDT',650],['GRTUSDT',1700],['DOTUSDT',10000],['ZILUSDT',1000],['FILUSDT',2000],['SCUSDT',200],['MATICUSDT',8000],['FTMUSDT',2000],['BANDUSDT',750],['1INCHUSDT',1000],['AUDIOUSDT',700]]
startdate = '2021-01-01'
enddate = '2022-02-22'
enddate = pd.to_datetime('today')

interval = '1h'
#easyband = pd.read_csv('easyband.csv')
easyband = pd.read_csv('allbands.csv')

#Columns=['Indicator','ALLTIME','CLOSERATIO','SMA100','SMA55']
#easyband = pd.DataFrame(easyband,columns=['Indicator','ATH','CLOSERATIO','RED','ORANGE','GREEN','Current'])
# Indicator,RED,ORANGE,GREEN,Current
easyband = pd.DataFrame(easyband,columns=['Indicator','RED','ORANGE','GREEN','Current'])

#easyband = easyband.loc[:,['Indicator','GREEN','GREENEASY']]

#EASY=float(easyband.loc[(easyband.Indicator=='BANDUSDT')].SMA100.values)
#print(easyband)
#EASY[0]
#easyband

In [36]:
entries = pd.read_csv('entries.csv')

entries = pd.DataFrame(entries, columns =['Indicator', 'lowest_entry']) 

#entries = pd.DataFrame(entrylist, columns =['Indicator', 'lowest_entry'],dtype = float) 
entries
entries.lowest_entry.astype(float)
indicator_list = entries['Indicator']
#indicator_list
#entries
entries.to_csv("entries.csv")

exposure = pd.DataFrame(exposurelist, columns =['Indicator', 'exposure'],dtype = float) 
exposure
exposure.exposure.astype(float)
#indicator_list
#entries
entries.to_csv("exposure.csv")

entries = entries.merge(exposure ,how="left",on="Indicator")
#entries

/Users/hjabbour/opt/anaconda3/envs/cryptodev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
#whitelist=whitelist.lstrip()
#whitelist = indicator_list
#array = whitelist.split(",")
array = indicator_list
wl = []
bl = []
blacklist = ""
whitelist = ""
symbols =[]
for i in range(0 ,len(array)):
    if (array[i].find('DOWN') != -1):
        bl.append(str(array[i]).lstrip())
        #print ("Contains given substring ")
        blacklist += str(array[i])
    elif (array[i].find('UP') != -1):
        bl.append(str(array[i]).lstrip())
        #print ("Contains given substring ")
        blacklist += str(array[i])

    else:
        wl.append(str(array[i]).lstrip().replace("/",""))
        whitelist +=str(array[i]).lstrip()
        #print ("Doesn't contains given substring")
    

    
#symbols to be used to get the prices from binance url
symbols = np.array(wl)


## get the data 
import os

live = pd.to_datetime('today')
start = pd.to_datetime(startdate)
end = pd.to_datetime(enddate)

root_url = 'https://api.binance.com/api/v1/klines'
symbol = 'STEEMETH'
interval = '1d'
url = root_url + '?symbol=' + symbol + '&interval=' + interval
print(url)


directory = 'BN-main'
table = []
   
        
        

for symbol in symbols:
        url = root_url + '?symbol=' + symbol + '&interval=' + interval
        data = json.loads(requests.get(url).text)
        dflive = pd.DataFrame(data ,columns =['open_time',
              'o', 'h', 'l', 'c', 'v',
              'close_time', 'qav', 'num_trades',
              'taker_base_vol', 'taker_quote_vol', 'ignore'])
        dflive.open_time = pd.to_datetime(dflive['open_time'],unit='ms')
        dflive=dflive.loc[(dflive.open_time>start)]
        lastrow = dflive.iloc[-1]
        now= float(lastrow.c)
        alth = dflive.h.max()
        altl = dflive.l.min()
        
    
        ## lowest entiry
        lowest = float(entries.loc[(entries.Indicator==symbol)].lowest_entry.values)
        #print(lowest)
        easyseries = easyband.loc[(easyband.Indicator==symbol)].GREEN.values
        #print(easyseries)
        if easyseries.size>0:
                green = easyseries[0]
        else:
                green = 0
        oeasyseries = easyband.loc[(easyband.Indicator==symbol)].ORANGE.values
        #print(easyseries)
        if oeasyseries.size>0:
                orange = oeasyseries[0]
        else:
                orange = 0
 
        tuple =(symbol,alth,now,lowest,green,orange)
        table.append(tuple)
        #print("name is: ",symbol ,alth,altl)
        

table[0]
AllTime = pd.DataFrame(table,columns=['Indicator','ATH','Current','LOWEST','GREEN','ORANGE'])
AllTime.ATH=AllTime.ATH.astype(float)
#AllTime.ATL=AllTime.ATL.astype(float)
AllTime.Current=AllTime.Current.astype(float)
AllTime.LOWEST=AllTime.LOWEST.astype(float)
AllTime.GREEN=AllTime.GREEN.astype(float)
AllTime.ORANGE=AllTime.ORANGE.astype(float)




AllTime.dtypes

#AllTime['ALLTIME']=(AllTime['Current']/AllTime['ATH'])*100
#AllTime['MONTH']=(AllTime['Current']/AllTime['OMH'])*100
#AllTime['2MONTH']=(AllTime['Current']/AllTime['TMH'])*100
#AllTime['3MONTH']=(AllTime['Current']/AllTime['THMH'])*100
AllTime['DROP']=((AllTime['Current']-AllTime['LOWEST'])/AllTime['LOWEST'])*100
AllTime['GDROP']=((AllTime['GREEN']-AllTime['Current'])/AllTime['GREEN'])*100
AllTime['TDROP']=((AllTime['DROP']*AllTime['GDROP']))

#AllTime.sort_values(by=['ALLTIME'],ascending=True,inplace = True)
#AllTime.sort_values(by='ATH',ascending=False)
#AllTime.to_csv("alltimeHL.csv", mode="w")
#data.sort_values("Name", axis = 0, ascending = True,inplace = True, na_position ='last')

## Display the Data 
AllTime.shape



https://api.binance.com/api/v1/klines?symbol=STEEMETH&interval=1d


(65, 9)

In [38]:
## Display the Data coins run last  Month

#drop = AllTime.loc[(AllTime['MONTH'] < 35)]
#drop = AllTime.loc[(AllTime['MONTH'] < 70)]
#drop = AllTime.loc[(AllTime['MONTH'] < 80)&(AllTime['2MONTH'] < 70)&(AllTime['ALLTIME'] < 40)]
drop = AllTime.loc[(AllTime['Current'] <= AllTime['LOWEST'])]

#sort = drop.sort_values(by=['EASY'],ascending=False,inplace = True)
sort = drop.sort_values(by=['GDROP','DROP'],ascending=[False,True],inplace = True)
sort = drop.sort_values(by=['TDROP'],ascending=True,inplace = True)

#drop = drop.loc[(drop['ALLTIME'] < drop['3MONTH'])]
display = drop[['Indicator','TDROP','GREEN','DROP','GDROP','LOWEST','Current']]

#print(display.iloc[0:60,0:14])
#print(display.iloc[60:110,])
#print(display.iloc[110:160,])
print("\b Fix Lowest entry ")

display = display.merge(exposure ,how="left",on="Indicator")
#display1 = display.sort_values(by=['exposure'],ascending=True,inplace = True)


display.to_csv("fixentries.csv")

display

 Fix Lowest entry 


/Users/hjabbour/opt/anaconda3/envs/cryptodev/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure
0,LUNAUSDT,-9999.948133,37.298096,-99.999615,99.999866,13.000000,0.000050,NaN
1,MIRUSDT,-5716.922827,0.686540,-89.429791,63.926380,2.343000,0.247660,85.0
2,PERPUSDT,-2779.067342,2.550476,-49.549550,56.086632,2.220000,1.120000,NaN
3,SUSHIUSDT,-2022.469613,1.945167,-62.428571,32.396538,3.500000,1.315000,2730.0
4,FILUSDT,-1388.764052,10.145476,-60.850000,22.822745,20.000000,7.830000,2000.0
5,RAYUSDT,-1284.577006,1.595429,-51.960000,24.722421,2.500000,1.201000,100.0
6,DENTUSDT,-1237.073158,0.001312,-58.031720,21.317189,0.002459,0.001032,385.0
7,LINAUSDT,-1223.812389,0.011672,-49.914724,24.518064,0.017590,0.008810,205.0
8,HNTUSDT,-1188.125983,11.434048,-41.366906,28.721654,13.900000,8.150000,999.0
9,TFUELUSDT,-1092.132461,0.080898,-51.076923,21.382111,0.130000,0.063600,244.0


## Green Drop


In [39]:
## Green Drop
display = display.loc[:]
#display = display.loc[display.DROP <= -30]
display.sort_values(by=['GDROP'],ascending=False,inplace = True)

#display = display.loc[display.Current < display.GREEN*.9]

display

,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure
0,LUNAUSDT,-9999.948133,37.298096,-99.999615,99.999866,13.000000,0.000050,NaN
1,MIRUSDT,-5716.922827,0.686540,-89.429791,63.926380,2.343000,0.247660,85.0
2,PERPUSDT,-2779.067342,2.550476,-49.549550,56.086632,2.220000,1.120000,NaN
3,SUSHIUSDT,-2022.469613,1.945167,-62.428571,32.396538,3.500000,1.315000,2730.0
8,HNTUSDT,-1188.125983,11.434048,-41.366906,28.721654,13.900000,8.150000,999.0
5,RAYUSDT,-1284.577006,1.595429,-51.960000,24.722421,2.500000,1.201000,100.0
7,LINAUSDT,-1223.812389,0.011672,-49.914724,24.518064,0.017590,0.008810,205.0
4,FILUSDT,-1388.764052,10.145476,-60.850000,22.822745,20.000000,7.830000,2000.0
9,TFUELUSDT,-1092.132461,0.080898,-51.076923,21.382111,0.130000,0.063600,244.0
6,DENTUSDT,-1237.073158,0.001312,-58.031720,21.317189,0.002459,0.001032,385.0


## Max entry Drop


In [40]:
## Max entry Drop
display = display.loc[:]
#display = display.loc[display.DROP <= -30]
display.sort_values(by=['DROP'],ascending=True,inplace = True)

#display = display.loc[display.Current < display.GREEN*.9]

display

,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure
0,LUNAUSDT,-9999.948133,37.298096,-99.999615,99.999866,13.000000,0.000050,NaN
1,MIRUSDT,-5716.922827,0.686540,-89.429791,63.926380,2.343000,0.247660,85.0
3,SUSHIUSDT,-2022.469613,1.945167,-62.428571,32.396538,3.500000,1.315000,2730.0
4,FILUSDT,-1388.764052,10.145476,-60.850000,22.822745,20.000000,7.830000,2000.0
42,CAKEUSDT,738.967754,3.968286,-60.373665,-12.239902,11.240000,4.454000,4000.0
6,DENTUSDT,-1237.073158,0.001312,-58.031720,21.317189,0.002459,0.001032,385.0
46,FLUXUSDT,inf,0.000000,-56.270519,-inf,1.523000,0.666000,224.0
11,THETAUSDT,-957.672985,1.519976,-55.142857,17.367127,2.800000,1.256000,NaN
15,BANDUSDT,-693.550863,1.907786,-53.833333,12.883298,3.600000,1.662000,750.0
33,SCUSDT,40.070874,0.005082,-53.454545,-0.749625,0.011000,0.005120,200.0


In [41]:
## Green entry
print("\b Enries on easy band")

display1 = display.loc[:]
display1

 Enries on easy band


,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure
0,LUNAUSDT,-9999.948133,37.298096,-99.999615,99.999866,13.000000,0.000050,NaN
1,MIRUSDT,-5716.922827,0.686540,-89.429791,63.926380,2.343000,0.247660,85.0
3,SUSHIUSDT,-2022.469613,1.945167,-62.428571,32.396538,3.500000,1.315000,2730.0
4,FILUSDT,-1388.764052,10.145476,-60.850000,22.822745,20.000000,7.830000,2000.0
42,CAKEUSDT,738.967754,3.968286,-60.373665,-12.239902,11.240000,4.454000,4000.0
6,DENTUSDT,-1237.073158,0.001312,-58.031720,21.317189,0.002459,0.001032,385.0
46,FLUXUSDT,inf,0.000000,-56.270519,-inf,1.523000,0.666000,224.0
11,THETAUSDT,-957.672985,1.519976,-55.142857,17.367127,2.800000,1.256000,NaN
15,BANDUSDT,-693.550863,1.907786,-53.833333,12.883298,3.600000,1.662000,750.0
33,SCUSDT,40.070874,0.005082,-53.454545,-0.749625,0.011000,0.005120,200.0


In [42]:
#display1['Value']=(display['DROP']/100)*display['exposure']
#display1

display = display1.loc[:]
#display['RED']=display['ORANGE']*2
display['Value']=(display['DROP']/100)*display['exposure']
sort = display.sort_values(by=['Value'],ascending=True,inplace = True)

#display['Ratio']=display['ATH']/display['Current']
#display['ORatio']=display['ORANGE']/display['Current']
#display = display.loc[display['Current'] <= display['GREEN']]
display


In [43]:
#del display ,sort, AllTime

In [44]:
#AllTime